# Generate instrument sector information using dedicated service - Python

### Overview
Industrial sector is an important information when looking for referential data. One of the challenges with sectors data remains in the various available classifications.
Sector classifications are organized in different levels to identify the different industries from the most general to the most specific.
The sector classifications currenctly supported are ICB, GICS, TRBC and SIC.

This sample requests **on-demand** sectors data by calling a dedicated service.

### Inputs/outputs
Sectors data sample is designed to help you requesting sectors data by using several classification types.
There are two ways to use the static sectors data endpoint. In both cases one of the necessary input is the **classification type**. The other input depends on the use-case:
* **code**: in order to check the meaning of a specific code (ex: '10101010')
* **level**: in order to retrieve a view of the table from a specific level. These levels correspond to layers of details of the sector activity.

The service returns the information in 3 different fields:
* **description**: a brief description of the classification.
* **count**: number of levels in the classification.
* **data**: array containing the elements retrieved


### Services used
This sample uses *gRPC requests* in order to retrieve sectors reference data from the hosted service. The queried endpoint in this script are:
* *StaticSectorService*: to directly retrieve sector information objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.sector_data.v1*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run Exchange data sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
import google.protobuf as pb
import systemathics.apis.services.static_data.v1.static_sector_pb2 as static_sector
import systemathics.apis.services.static_data.v1.static_sector_pb2_grpc as static_sector_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Define necessary functions
In this part we define the functions dedicated to retrieve **sectors data**.
These snippets will be used in the next paragraphs for examples

In [ ]:
#define a method to get sectors information for a specific code
def get_sector_data_code(provider, code):
    
    # generate sector data request
    request = static_sector.StaticSectorRequest( 
        provider = provider,
        code = code
    )
    
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:  

            # instantiate the sector data service
            service = static_sector_service.StaticSectorServiceStub(channel)

            # process the request
            response = service.StaticSector(
                request = request, 
                metadata = [('authorization', token)]
            )
        return response
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())

In [ ]:
#define a method to get sectors information for a level
def get_sector_data_level(provider, level):
    
    # generate sector data request
    request = static_sector.StaticSectorRequest( 
        provider = provider,
        level = level
    )
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:  

            # instantiate the sector data service
            service = static_sector_service.StaticSectorServiceStub(channel)

            # process the request
            response = service.StaticSector(
                request = request, 
                metadata = [('authorization', token)]
            )
        return response
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())    

### Step 4: Retrieve sector classification from a level
In this sample we retrieve the classification for a specific level

#### 4.1: First example: level 0 of ICB

In [ ]:
provider = 'ICB'
level = 0
data = get_sector_data_level(provider, level)

In [ ]:
print(data.description)
print("Number of levels: {}".format(data.count))

In [ ]:
#pPrepare the data frame content
names = [b.Name for b in data.data]
codes = [b.Code for b in data.data]
labels = [b.label for b in data.data]

d = {'Names': names, 'Codes': codes, 'Labels': labels}
df = pd.DataFrame(data=d)
df

#### 4.2: Second example: level 3 of GICS

In [ ]:
provider = 'GICS'
level = 3
data = get_sector_data_level(provider, level)

In [ ]:
print(data.description)
print("Number of levels: {}".format(data.count))

In [ ]:
#pPrepare the data frame content
names = [b.Name for b in data.data]
codes = [b.Code for b in data.data]
labels = [b.label for b in data.data]

d = {'Names': names, 'Codes': codes, 'Labels': labels}
df = pd.DataFrame(data=d)
df

### Step 5: Retrieve sectors data from a specific code
In this part we assume we have the code for a specific classification and we want to get the corresponding sector 

#### 5.1: ICB

In [ ]:
# set the exchange to retrieve equities data
provider = 'ICB'
code = '30201025'

In [ ]:
icb = get_sector_data_code(provider, code)
icb.data

#### 5.2: SIC

In [ ]:
# set the exchange to retrieve equities data
provider = 'SIC'
code = '3577'

In [ ]:
sic = get_sector_data_code(provider, code)
sic.data

#### 5.3: GICS

In [ ]:
# set the exchange to retrieve equities data
provider = 'GICS'
code = '10102010'

In [ ]:
gics = get_sector_data_code(provider, code)
gics.data

#### 5.4: TRBC

In [ ]:
# set the exchange to retrieve equities data
provider = 'TRBC'
code = '5020101010'

In [ ]:
trbc = get_sector_data_code(provider, code)
trbc.data